In [129]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [130]:
# Membaca file CSV
file_path = 'data beras jawa barat.csv'
df = pd.read_csv(file_path)

In [131]:
# Membaca file CSV dengan titik koma sebagai pemisah
df = pd.read_csv(file_path, delimiter=';')

# Menampilkan 5 baris pertama dari data yang sudah dibaca
df.head()

,Pasar,1/12/2023,1/13/2023,1/14/2023,1/15/2023,1/16/2023,1/17/2023,1/18/2023,1/19/2023,1/20/2023,...,10/22/2023,10/23/2023,10/24/2023,10/25/2023,10/26/2023,10/27/2023,10/28/2023,10/29/2023,10/30/2023,10/31/2023
0,Jawa Barat,12500,12500,12000,12500,12500,12500,12500,12500,12500,...,14140,14130,14160,14140,14110,14200,14160,14170,14150,14170
1,Kab. Bogor,12000,12000,12000,-,12000,12000,12000,12000,-,...,15500,15500,15500,14500,14500,14500,14500,14500,14500,14500
2,Kab. Sukabumi,12000,12000,12000,12000,12000,12000,12000,12000,12000,...,14000,14000,14000,14000,14000,14000,14000,14000,14000,14000
3,Kab. Cianjur,12500,12500,12500,12500,12500,13000,13000,13000,13000,...,14170,14170,14170,14170,14170,14170,14170,14170,14170,14170
4,Kab. Bandung,12000,12000,12000,12500,12000,12000,12000,12000,12000,...,13500,13500,14000,13500,13500,14500,14500,14500,14000,14500


In [132]:
# Mengubah data dari format lebar ke format panjang
df_long = df.melt(id_vars=["Pasar"], 
                  var_name="Tanggal", 
                  value_name="Harga")

# Mengubah kolom Tanggal menjadi tipe datetime
df_long["Tanggal"] = pd.to_datetime(df_long["Tanggal"], format='%m/%d/%Y')

# Mengubah kolom Harga menjadi tipe numerik
df_long["Harga"] = pd.to_numeric(df_long["Harga"], errors='coerce')

# Menampilkan 5 baris pertama dari data yang sudah diubah
df_long.head()

,Pasar,Tanggal,Harga
0,Jawa Barat,2023-01-12,12500.0
1,Kab. Bogor,2023-01-12,12000.0
2,Kab. Sukabumi,2023-01-12,12000.0
3,Kab. Cianjur,2023-01-12,12500.0
4,Kab. Bandung,2023-01-12,12000.0


In [133]:
# Menggunakan data yang sudah diubah ke format panjang
X = df_long[['Tanggal']]
y = df_long['Harga']

# Membagi data menjadi data pelatihan dan data pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [134]:
print("Jumlah nilai NaN dalam kolom Harga:", y_train.isna().sum())

Jumlah nilai NaN dalam kolom Harga: 209


In [136]:
# Mengelompokkan data berdasarkan tanggal dan menghitung rata-rata harga
df_long_grouped = df_long.groupby('Tanggal')['Harga'].mean()

# Fungsi untuk mengisi nilai NaN dengan rata-rata harga pada hari itu
def isi_na_dengan_rata_rata(row):
    if pd.isnull(row['Harga']):
        return df_long_grouped[row['Tanggal']]
    return row['Harga']

# Mengisi nilai NaN dalam kolom Harga dengan rata-rata harga pada hari itu
df_long['Harga'] = df_long.apply(isi_na_dengan_rata_rata, axis=1)

# Menghapus baris yang masih berisi NaN setelah pengisian nilai NaN
df_long.dropna(inplace=True)



In [137]:
# Lanjutkan dengan pelatihan model
X = df_long[['Tanggal']]
y = df_long['Harga']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inisialisasi dan melatih model Random Forest dengan penyetelan parameter
model = RandomForestRegressor(n_estimators=200, max_depth=10, min_samples_split=5, random_state=0)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, min_samples_split=5, n_estimators=200,
                      random_state=42)

In [138]:
# Memprediksi harga untuk 1 bulan ke depan
last_date = df_long['Tanggal'].max()
next_month = last_date + pd.DateOffset(months=1)
future_dates = pd.date_range(start=next_month, periods=30, freq='D')
future_df = pd.DataFrame({'Tanggal': future_dates})
future_df['Harga'] = model.predict(future_df[['Tanggal']])

# Menampilkan data prediksi untuk 1 bulan ke depan
print(future_df)


      Tanggal         Harga
0  2023-11-30  14178.560191
1  2023-12-01  14178.560191
2  2023-12-02  14178.560191
3  2023-12-03  14178.560191
4  2023-12-04  14178.560191
5  2023-12-05  14178.560191
6  2023-12-06  14178.560191
7  2023-12-07  14178.560191
8  2023-12-08  14178.560191
9  2023-12-09  14178.560191
10 2023-12-10  14178.560191
11 2023-12-11  14178.560191
12 2023-12-12  14178.560191
13 2023-12-13  14178.560191
14 2023-12-14  14178.560191
15 2023-12-15  14178.560191
16 2023-12-16  14178.560191
17 2023-12-17  14178.560191
18 2023-12-18  14178.560191
19 2023-12-19  14178.560191
20 2023-12-20  14178.560191
21 2023-12-21  14178.560191
22 2023-12-22  14178.560191
23 2023-12-23  14178.560191
24 2023-12-24  14178.560191
25 2023-12-25  14178.560191
26 2023-12-26  14178.560191
27 2023-12-27  14178.560191
28 2023-12-28  14178.560191
29 2023-12-29  14178.560191


In [139]:
# Menampilkan prediksi harga tertinggi dan terendah untuk 1 bulan ke depan
harga_tertinggi_1_bulan = max(future_df['Harga'])
harga_terendah_1_bulan = min(future_df['Harga'])
print("Prediksi Harga Tertinggi untuk 1 Bulan ke Depan:", harga_tertinggi_1_bulan)
print("Prediksi Harga Terendah untuk 1 Bulan ke Depan:", harga_terendah_1_bulan)

Prediksi Harga Tertinggi untuk 1 Bulan ke Depan: 14178.560191455315
Prediksi Harga Terendah untuk 1 Bulan ke Depan: 14178.560191455315


In [140]:
# Membuat kolom prediksi klasifikasi
threshold = (harga_tertinggi_1_bulan + harga_terendah_1_bulan) / 2
future_df['Prediksi_Klasifikasi'] = np.where(future_df['Harga'] >= threshold, 1, 0)

# Membuat kolom klasifikasi aktual berdasarkan harga rata-rata sebelumnya
average_price = df_long['Harga'].mean()
df_long['Klasifikasi_Aktual'] = np.where(df_long['Harga'] >= average_price, 1, 0)

# Evaluasi kinerja model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score

# Matriks Konfusi
cm = confusion_matrix(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("Confusion Matrix:")
print(cm)

# Akurasi
accuracy = accuracy_score(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("Akurasi:", accuracy)

# Presisi
precision = precision_score(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("Presisi:", precision)

# F1-Score
f1score = f1_score(df_long['Klasifikasi_Aktual'], np.where(df_long['Harga'] >= threshold, 1, 0))
print("F1-Score:", f1score)

Confusion Matrix:
[[3943    0]
 [3512  441]]
Akurasi: 0.5552178318135765
Presisi: 1.0
F1-Score: 0.20072826581702322
